$\qquad$ $\qquad$$\qquad$  **TDA 231 Machine Learning: Homework 2** <br />
$\qquad$ $\qquad$$\qquad$ **Goal: Get acquainted with different classification techniques.**<br />
$\qquad$ $\qquad$$\qquad$                   **Grader: Victor Eberstein** <br />
$\qquad$ $\qquad$$\qquad$                   **Submitted by: Name, Personal no., email** <br />

---

General guidelines:
*   All solutions to theoretical and pratical problems must be submitted in this ipynb notebook, and equations wherever required, should be formatted using LaTeX math-mode.
*   All discussion regarding practical problems, along with solutions and plots should be specified in this notebook. All plots/results should be visible such that the notebook do not have to be run. But the code in the notebook should reproduce the plots/results if we choose to do so.
*   Your name, personal number and email address should be specified above.
*   All tables and other additional information should be included in this notebook.
*   All the answers for theoretical questions must be filled in the cells created for you with "Your answer here" below each question, but feel free to add more cells if needed.
*   Before submitting, make sure that your code can run on another computer. That all plots can show on another computer including all your writing. It is good to check if your code can run here: https://colab.research.google.com.

### General comments:
* **Throughout this assignment, feel free to use any *NumPy*-functions where you see fit.**
* **For the practical tasks there are comments marked with "TODO" in all parts that you need to change.**
* **Please note the slight changes in the notation from the lectures:**
    * Here, the dataset is represented as $\{ (\mathbf{x}_n, y_n)\}_{n=1}^N$, where $x_n$ are vectors with attributes of dimension $D$ and $y_n$ is the associated labels ($y_n \in \{0,1\}$ for binary prediction). In the lectures, $t_n$ was instead used for the labels.

# 0. Dataset
**Run the code cell below to produce the dataset and some helper functions.**

In [ ]:
from sklearn.datasets import make_gaussian_quantiles
import numpy as np
from matplotlib import pyplot as plt

# Randomly generate data from two different distributions
data_class1, _ = make_gaussian_quantiles(mean = [1,1], cov = 1.5, n_samples = 30, n_features=2, random_state=18)
data_class2, _ = make_gaussian_quantiles(mean = [-1,-1], cov= 0.75,n_samples = 8, n_features=2, random_state=4)

# Concatenate the data, and add labels
X_train = np.append(data_class1, data_class2, axis=0)
y_train = np.append(np.zeros(len(data_class1), dtype=np.int), 
                   np.ones(len(data_class2), dtype=np.int))

# Consider new test point
X_test = np.array([[-0.6,-0.4]])

def plot_data(X_train, y_train, X_test, title = ''):
    # Plot the two data classes
    fig,ax = plt.subplots(figsize=(10,10))
    ax.plot(X_train[y_train == 0][:,0], X_train[y_train == 0][:,1], 'o', markersize=8)
    ax.plot(X_train[y_train == 1][:,0], X_train[y_train == 1][:,1], 's', markersize=8)

    # Plot test point (circles to help with distances)
    ax.plot(X_test[:,0], X_test[:,1], '*', markersize=10)

    plt.axis('square')
    ax.legend(['Class 0','Class 1','test point'])
    ax.grid()
    ax.set_title(title, fontsize=15);
    return fig, ax

def decision_boundary(model, fig, ax, levels = [0.1,0.5,0.9], labels = True):
    # Code for producing the plot
    X1 = np.linspace(*ax.get_xlim(),100)
    X2 = np.linspace(*ax.get_ylim(), 100)
    Z = np.zeros(X1.shape+X2.shape)

    for i,x1 in enumerate(X1):
        for j,x2 in enumerate(X2):
            _, y_prob = model.predict(np.array([x1,x2]).reshape(1,-1))
            Z[j,i] = y_prob[0,0]
            
    contours = ax.contour(X1,X2,Z, levels=levels)
    if labels:
        ax.clabel(contours, inline=1, fontsize=10)
        
    return fig, ax

# 1. K-Nearest-Neighbour Classification (3 pts)
A KNN classifier assigns a test instance to the majority class associated with its $K$ nearest training instances. Distance between instances is measured using Euclidean distance. Suppose we have the following training set of two classes (circles and squares) and a single test instance (star). Assume “unweighted” KNN (every nearest neighbor contributes equally to the final vote).

**Run the code in the cell below.**

In [ ]:
fig, ax = plot_data(X_train, y_train, X_test)

# Circles added to help visualize distance to points.
for r in np.arange(0.25,1,0.25):
    c = plt.Circle(X_test[0], r, color='g', fill=False)
    ax.add_artist(c)

### 1.1 Exercise 1 (1 pts)
a) What would be the class assigned to this test instance for $K=2$, $K=4$, $K=5$ and why? (**0.5 pt**)

b) The classification result is affected by the increasing $K$. What will be the maxinum value of K in this case? Why? (**0.5 pt**)<br />
(***Hint:*** After $K$ reaches a certain value, it is impossible for the classification results to change. Find this value!)


#### Your answer here:

### 1.2 Exercise 2 (2 pts)
In this task, we are going to complete a custom implementation of the unweighted K-Nearest-Neighbor classifier. A skeleton of the implementation can be found below, and the only function that is not finished is `predict(self, X_test)`.

In total, the algorithm stores three variables:
* `self.X_train`, `self.y_train`: All training data with labels.
* `self.n_neighbors`: Number of neighbors, i.e. $K$.

Your task is to finish the implementation of `predict(self, X_test)`. The distances between all test- and training instances have already been computed. In the loop below, the variable `dist_i` is an array containing the distances between test sample $i$ and all training points in `self.X_train`. Your task is to compute `y_pred_prob[i]`, which is an array with the probability for selecting each class in the dataset, e.g. for binary prediction we have $[1,0]$ (class 0), $[0,1]$ (class 1), or $[0.5,0.5]$ (tie).

Finish this implementation and verify your results in Exercise 1 by running the implementation for the aforementioned values of $K$ (2,4, and 5).
<br />
***Hint:*** `np.unique` might be of interest.

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
class K_Nearest_Neighbors(): 
    """
    Custom implementation of K-nearest-neighbors.
    
    Note: If two or more classes have the same amount of neighbors, the prediction 
    will be random, i.e. tiebreaks are resolved randomly.
    """
    
    def __init__(self, n_neighbors):
        """
        Parameter(s):
            n_neighbors - Number of neighbors
        """
        self.n_neighbors = n_neighbors
        
    def fit(self, X_train, y_train):
        """
        No training required. Store data and labels.
        --------------------
        Input(s):
            X_train   - Data of size (n_samples, n_features)
            y_train   - True labels of size (n_samples,1)
        --------------------
        Output(s)
        """
        self.X_train = X_train
        self.y_train = y_train
        
    def predict(self, X_test):
        """
        Prediction of test data.
        --------------------
        Input(s):
            X_test   - Data of size (n_samples, n_features)
        --------------------
        Output(s)
            y_pred - Predicted labels of size (n_samples,)
            y_pred_prob - Probabilistic labels of size (n_samples,n_classes)
        """
        n_samples = X_test.shape[0]
        unique_classes = np.unique(self.y_train)
        n_classes = len(unique_classes)

        y_pred = np.zeros(shape=(n_samples,), dtype=np.int32)
        y_pred_prob = np.zeros(shape=(n_samples, n_classes), dtype=np.float32)
        
        # Computes distances between all points in X_test and X_train.
        dist = euclidean_distances(X_test,self.X_train)
        
        for i in range(n_samples):
            dist_i = dist[i]
            
            ### TODO - Change this part
            y_pred_prob[i,:] = [1,0] # Always predicts class 0 currently
            ### 
            
            # Selects the prediction randomly, based on y_pred_prob[i]. 
            y_pred[i] = np.random.choice(unique_classes, p=y_pred_prob[i])
        
        return y_pred, y_pred_prob

#### Run the code below after finishing the above implementation (no need to change anything).
Make sure that you understand the plot and that your implementation produces sensible results.

*Note:* What happens when $K$ is even?

In [ ]:
# Parameters (feel free to change)
K = 3

# Define and train the model
model = K_Nearest_Neighbors(n_neighbors = K)
model.fit(X_train, y_train)

# Predictions
y_pred_train,_ = model.predict(X_train)
print("Train accuracy: %.3f %%" %(np.mean(y_pred_train == y_train)*100))
y_pred_test,_ = model.predict(X_test)
print("Predicted test class, K=%d: %d" %(K, y_pred_test))

# Plot decision-boundaries
fig, ax = plot_data(X_train, y_train, X_test, title = 'Lines showing decision-boundaries')
fig, ax = decision_boundary(model, fig, ax, levels=[0.4, 0.6], labels = False)

# 2. Naive Bayes Classifier (4 pts)

The Bayes classifier refers to a classifier using the predicitive probability:
$$
P(y_{n} = c | x_{n}, X, y) = \frac{p(x_{n}|y_{n} = c, X, y)P(y_{n} = c | X, y)}{p(x_{n}|X,y)}, \tag{1}
$$
where $X,y$ is the training data, $x_{n}$ a new test instance and $c$ its corresponding class.

Assume that the class-conditional densities are Gaussian, that is
$$
    p(x_{n}|y_{n} = c, X, y) = \mathcal{N}(\mu_{c}, \Sigma_{c}).
$$

Furthermore, assume a class size prior, i.e. the prior is set using the proportion of training points in class $c$:
$$
    P(y_{n} = c |X, y ) = \frac{N_c}{N}, \tag{2}
$$
where $N_c$ is the number of training objects belonging to class $c$, and $N$ is the total number of training objects.


### Exercise  2.1 (3 pts)
We consider a **binary** classification problem with **real valued data**, i.e. $x_n = [x_{n1}, x_{n2}]^T \in \mathbb{R}^2$ and $c=\{0,1\}$.


#### 1.  (1 pts)
In order to simplify the problem, we use the Naive Bayes assumption,
$$
    p(x_{n}|y_{n} = c, X, y) = \prod_{d=1}^2 p(x_{nd}|y_n = c, X, y) = \prod_{d=1}^2 \mathcal{N}(\mu_{cd}, \sigma_{cd}^2),
$$
which assumes that the two attributes of $x_n$ are independent of one another, i.e. the covariance matrix $\Sigma_{c}$ is diagonal.

Using this assumption, write the expression for the **naive Bayes** classifier, that is, derive
$$
P(y_{n} = c | x_{n} , X, y ).
$$

***HINT:*** Derive the maximum likelihood estimates for the parameters $\mu_{c}, \Sigma_{c}$. Then express eq. 1 in terms of those estimates. 

#### 2. (2 pts)
Derive the maximum likelihood estimate for $\mu_{c}, \Sigma_{c}$ when the covariance matrix is not diagonal, i.e, $\Sigma_{c}$ has 4 unknown scalars. This alleviates the "naive" assumption, since the feature components are no longer independent of one another.

***HINT:***
When deriving the MLE for $\Sigma_{c}$ it might be a good idea to consider $\partial /\partial \Sigma_{c}^{-1}$. Also, feel free to use the following properties:
$$
    \frac{\partial\ x^T A x}{\partial A} = x x^T, \qquad
    \frac{\partial\ x^T A x}{\partial x} = 2Ax, \qquad
    \frac{\partial\ \log |A|}{\partial A} = A^{-T},
$$
for a symmetric matrix $A$ and a vector $x$ that does not depend on $A$.

#### Your answer here:

### Exercise  2.2 (1 pts)
Here, you will finish a custom implementation of a Gaussian Naive Bayes classifier.

The most important variables stored by the algorithm are:
* `self.mean`, and `self.std`: Dictionaries containing MLE for each class. (`self.mean[c]`$ =[\mu_{c1}, \mu_{c2}]$, and `self.std[c]`$ =[\sigma_{c1}, \sigma_{c2}]$)
* `self.prior`: Dictionary containing prior probability of each class. 

Use your calculations from Exercise 2.1.1 and complete the unfinished function `predict(self, X_test)`. The 1-D Gaussian density function `gaussian_density(self, x, mu, std)` is already implemented and may be used.

In [ ]:
class GaussianNaiveBayes(): 
    """
    Custom implementation of a Gaussian Naive Bayes classifier.
    The parameters are estimated using MLE.
    """
    
    def __init__(self):
        """
        Parameter(s):
        """
        self.mean = {}
        self.std = {}
        self.prior = {}
        self.unique_classes = 0
        
    def get_class_parameters(self, X_class):
        """
        Estimating the MLE of the parameters.
        --------------------
        Input(s):
            X_class - Data points corresponding to a single class
        --------------------
        Output(s):
            mean_MLE - MLE of mean
            std_MLE  - MLE of scale
        """
        
        mean_MLE = np.mean(X_class, axis=0)
        std_MLE = np.std(X_class, axis=0)
        
        return mean_MLE, std_MLE
            
    def fit(self, X_train, y_train):
        """
        Compute model parameters using maximum likelihood estimates and a class size prior.
        --------------------
        Input(s):
            X_train   - Data of size (n_samples, n_features)
            y_train   - True labels of size (n_samples,1)
        --------------------
        Output(s)
        """
        
        # Compute mean, variance and prior of each class
        self.unique_classes = np.unique(y_train)
        for uc in self.unique_classes:
            X_class = X_train[y_train == uc]
            c_mean, c_std = self.get_class_parameters(X_class)
            self.mean[uc] = c_mean
            self.std[uc] = c_std
            self.prior[uc] = X_class.shape[0]/X_train.shape[0]
            
    def gaussian_density(self, x, mu, std):
        """
        1-D Gaussian density function.
        --------------------
        Input(s):
            x   - Data point
            mu  - mean
            std - scale
        --------------------
        Output(s):- 
            N(mu, std^2)

        """
        return 1/(std*np.sqrt(2*np.pi))*np.exp(-(1/2)*((x-mu)/std)**2)

    def predict(self, X_test):
        """
        Prediction of test data.
        --------------------
        Input(s):
            X_test   - Data of size (n_samples, n_features)
        --------------------
        Output(s)
            y_pred - Predicted labels of size (n_samples,)
            y_pred_prob - Probabilistic labels of size (n_samples,n_classes)
        """
        
        n_samples = X_test.shape[0]
        y_pred_prob = np.zeros(shape=(n_samples,len(self.unique_classes)), dtype=np.float32)
        
        ### TODO - Change this part
        for i in range(n_samples):
            y_pred_prob[i,:] = [1,0] # Always predicts class 0 currently
        ###
        
        
        y_pred = np.argmax(y_pred_prob, axis=-1)
        return y_pred, y_pred_prob

#### Run the code below after finishing the above implementation (no need to change anything).
Make sure that you understand the plot and that your implementation produces sensible results.

In [ ]:
# Define and train the model
model = GaussianNaiveBayes()
model.fit(X_train, y_train)

# Prediction
y_pred_train, _ = model.predict(X_train)
print("Train accuracy: %.3f %%" %(np.mean(y_pred_train == y_train)*100))
y_pred_test,y_pred_prob_test = model.predict(X_test)
print("Predicted test class: %d, (prob %.2f)" %(y_pred_test, y_pred_prob_test[0,y_pred_test]))

# Plot decision-boundaries
fig, ax = plot_data(X_train, y_train, X_test, title = 'Lines showing $P(y_{n} = 0 | x_{n}, X, y)$')
fig, ax = decision_boundary(model, fig, ax, labels = True)

# 3. SVM (4 pts)


### Excercise 3.1 (2 pts)
The SVM classifier is most often phrased as the constrained optimization problem:
$$
\begin{aligned}
 \underset{w,b}{\arg\min}\ & \frac{1}{2} w^T w \\
 s.t. \quad & t_n(w^T x_n + b) \geq 1, \forall n,
 \end{aligned}
$$
where $t_n \in \{-1,1\}$ are now the associated labels ($y_n = 0 \iff t_n = -1$; $y_n = 1 \iff t_n = 1$).

By moving the constraints into the objective function, we may formulate the problem using the *hinge* loss function:
$$
    J =  \frac{1}{2} w^T w + C\frac{1}{n} \sum_{i=1}^n \max(0, 1 - t_n(w^T x_n + b)),
$$
with $C$ as a regularization parameter.

Even though the *hinge* loss function is not differentiable, it is still a convex function and has subgradients with respect to both $w$ and $b$. Thus, it can still be used for optimization.
Derive the following subgradients:
$$
\frac{\partial J}{\partial w}, \quad
\frac{\partial J}{\partial b}.
$$

#### Your answer here:

### Excercise 3.2 (2 pts)
Here, you will finish a custom implementation of a SVM classifier.

The most important variables stored by the algorithm are:
* `self.C`: Regularization parameter.
* `self.parameters`: Dictionary containing the model parameters $w$ and $b$.

Using the subderivatives derived in Exercise 3.1, complete the unfinished function `grad_cost(self, X_train, t, w, b)`, which should return the cost functions and the derivatives of the cost function w.r.t $w,b$.

Also, finish the function `predict(self, X_test)` using the appropriate prediction strategy for SVM.

In [ ]:
class SVM(): 
    """
    Custom implementation of linear SVM.
    """
    def __init__(self, learning_rate, n_iterations, C = 1, print_cost=False):
        """
        Parameter(s):
            learning_rate - Learning rate
            n_iterations  - Number of iterations
            C             - Regularization parameter
        """
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations 
        self.C = C
        self.print_cost = print_cost
        self.parameters = {}
    
    def initialize_parameters(self, n_features): 
        """
        Initialize model parameters with zeros:
            w.shape = (n_features, 1)
            b.shape = (1,)
        --------------------
        Input(s):
            n_features - Number of features 
        --------------------
        Output(s):
        """

        w = np.zeros(shape = (n_features,1))
        b = 0

        self.parameters["w"] = w
        self.parameters["b"] = b

    
    def grad_cost(self, X_train, t, w, b):
        """
        Computes the cost function (hinge loss) and 
        partial derivatives of cost w.r.t the model parameters.
        --------------------
        Input(s):
            X_train - Data of size(n_samples, n_features)
            t       - Labels (-1 or 1) of size (n_samples,)
            w,b     - Model parameters
        --------------------
        Output(s):
            cost  - hinge loss
            grads - Gradients of loss function w.r.t model parameters (dw,db). 
        """
        n_samples = X_train.shape[0]
        
        ### TODO - Change this part
        # Dummy variables currently
        cost = 0
        dw = 0
        db = 0
        ### 
        
        grads = {"dw": dw,
                 "db": db}

        return cost, grads
        
    def fit(self, X_train, y_train): 
        """
        Optimize model parameters by running a gradient descent algorithm.
        --------------------
        Input(s):
            X_train - Data of size (n_samples, n_features)
            y_train - True labels of size (n_samples,1)
        --------------------
        Output(s)
        """
        n_features = X_train.shape[1]
        
        # Change the binary targets from (0,1) to (-1,1)
        t = np.where(y_train < 0.5, -1, 1)
        
        # Initialize parameters
        self.params = self.initialize_parameters(n_features)
        
        # Gradient descent
        w = self.parameters["w"] 
        b = self.parameters["b"] 
        for i in range(1,self.n_iterations+1):
            cost, grads = self.grad_cost(X_train, t, w, b)
            dw = grads["dw"]
            db = grads["db"]

            w -= self.learning_rate * dw
            b -= self.learning_rate * db 
            
            if self.print_cost and i % 100 == 0:
                print("Cost after iteration %i: %f" %(i, cost))

        self.parameters = {"w": w,
                           "b": b}   
        
        return self

    def predict(self, X_test):
        """
        Binary prediction of test data.
        --------------------
        Input(s):
            X   - Data of size (n_samples, n_features)
        --------------------
        Output(s)
            y_pred - Predicted labels (0/1) of size (n_samples,1)
        """
        
        w = self.parameters["w"]
        b = self.parameters["b"]
        
        n_samples = X_test.shape[0]
        y_pred = np.zeros((n_samples,1))
        
        ### TODO - Change this part
        for i in range(n_samples):
            y_pred[i] = -1 # Always predicts class 0 currently
        ###    
        
        # Converts the binary targets from (-1,1) to (0,1)
        y_pred = (y_pred > 0).astype(np.int32)
        
        return y_pred.squeeze()

#### Run the code below after finishing the above implementation (no need to change anything).
Make sure that you understand the plot and that your implementation produces sensible results.

In [ ]:
# Parameters (feel free to change)
learning_rate = 0.01
n_iterations = 1000
C = 1
print_cost = True

# Define and train the model
model = SVM(learning_rate, n_iterations, C, print_cost)
model.fit(X_train, y_train)

# Predictions
y_pred_train = model.predict(X_train)
print("Train accuracy: %.3f %%" %(np.mean(y_pred_train == y_train)*100))
y_pred_test = model.predict(X_test)
print("Predicted test class: %d" %(y_pred_test))

# Plot decision-boundaries
fig, ax = plot_data(X_train, y_train, X_test, title = 'Line showing decision boundary')
w = model.parameters['w']
b = model.parameters['b']
x = np.linspace(-3,3)
ax.plot(x, (-b-w[0]*x)/w[1], '-r')

# 4. Logistic Regression (2 pts)

### Excercise 4.1 (2 pts)
The last classifier that we will look at is Logistic Regression. Here you will finish a custom implementation of the Logistic Regression classifier using the following information:

* The activation is computed as 
$$
A = \sigma(X w + b) = [a_1, \ldots, a_n]^T,
$$
where $\sigma(z)$ is the sigmoid-function.

* The cost function is the negative log-likelihood:
$$
J = -\frac{1}{n}\sum_{i=1}^{n}y_i\log(a_i)+(1-y_i)\log(1-a_i).
$$

* Derivatives w.r.t the model parameters:
$$
 \frac{\partial J}{\partial w} = \frac{1}{n}X^T(A-Y), \qquad
\frac{\partial J}{\partial b} = \frac{1}{n} \sum_{i=1}^n (a_i-y_i).
$$

Finish the implementation of `grad_cost(self, X_train, y_train, w, b)`, which once again should return the cost functions and the derivatives of the cost function w.r.t $w,b$.

Also, finish the function `predict(self, X_test)` using the appropriate prediction strategy for Logistic Regression.

In [ ]:
class LogisticRegression(): 
    """
    Custom implementation of (binary) Logistic Regression.
    """
    def __init__(self, learning_rate, n_iterations, print_cost=False):
        """
        Parameter(s):
            learning_rate - Learning rate
            n_iterations  - Number of iterations
        """
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations 
        self.print_cost = print_cost
        self.parameters = {}
    
    def initialize_parameters(self, n_features): 
        """
        Initialize model parameters with zeros:
            w.shape = (n_features,)
            b.shape = (1,)
        --------------------
        Input(s):
            n_features - Number of features 
        --------------------
        Output(s):
        """
        
        w = np.zeros(shape = (n_features,1))
        b = 0
        
        self.parameters['w'] = w
        self.parameters['b'] = b
        
    def sigmoid(self, z):
        """
        Sigmoid function
        --------------------
        Input(s):
            z - 
        --------------------
        Output(s):
            s - sigmoid(z)
        """
        s = 1 / (1 + np.exp(-z))
        return s
    
    def grad_cost(self, X_train, y_train, w, b):
        """
        Computes the cost function (negative log-likelihood) and 
        partial derivatives of cost w.r.t the model parameters.
        --------------------
        Input(s):
            X_train - Data of size(n_samples, n_features)
            y_train - True labels of size (n_samples,1)
            w,b     - Model parameters
        --------------------
        Output(s):
            cost  - Negative log-likelihood cost
            grads - Gradients of loss function w.r.t model parameters (dw,db). 
        """
        n_samples = X_train.shape[0]
        
        y_train = y_train.reshape(-1,1)
        
        ### TODO - Change this part
        # Dummy variables currently
        A = 0
        cost = 0
        dw = 0
        db = 0
        ###

        grads = {"dw": dw,
                 "db": db}
        
        return cost, grads
        
    def fit(self, X_train, y_train): 
        """
        Optimize model parameters by running a gradient descent algorithm.
        --------------------
        Input(s):
            X_train - Data of size (n_samples, n_features)
            y_train - True labels of size (n_samples,1)
        --------------------
        Output(s)
        """
        n_features = X_train.shape[1]
        
        # Initialize parameters
        self.params = self.initialize_parameters(n_features)
        
        # Gradient descent
        w = self.parameters["w"] 
        b = self.parameters["b"] 
        for i in range(1,self.n_iterations+1):
            cost, grads = self.grad_cost(X_train, y_train, w, b)
            dw = grads["dw"]
            db = grads["db"]

            w -= self.learning_rate * dw
            b -= self.learning_rate * db 
            
            if self.print_cost and i % 100 == 0:
                print("Cost after iteration %i: %f" %(i, cost))

        self.parameters = {"w": w,
                           "b": b}   
        
        return self

    def predict(self, X_test):
        """
        Binary prediction of test data.
        --------------------
        Input(s):
            X   - Data of size (n_samples, n_features)
        --------------------
        Output(s)
            y_pred - Predicted labels (0/1) of size (n_samples,)
        """
        
        w = self.parameters["w"]
        b = self.parameters["b"]
        
        n_samples = X_test.shape[0]
        y_pred = np.zeros((n_samples,1))
        
        ### TODO - Change this part
        A = np.zeros((n_samples,1))                
        for i in range(n_samples):
            y_pred[i] = 0 # Always predicts class 0 currently
        ###
        
        return y_pred.squeeze(), A

#### Run the code below after finishing the above implementation (no need to change anything).
Make sure that you understand the plot and that your implementation produces sensible results.

In [ ]:
# Parameters (feel free to change)
learning_rate = 0.05
n_iterations = 1000
print_cost = True

# Define and train the model
model = LogisticRegression(learning_rate, n_iterations, print_cost)
model.fit(X_train, y_train)

# Predictions
y_pred_train, _ = model.predict(X_train)
print("Train accuracy: %.3f %%" %(np.mean(y_pred_train == y_train)*100))
y_pred_test,y_pred_prob_test = model.predict(X_test)
print("Predicted test class: %d, (prob %.2f)" %(y_pred_test, np.where(y_pred_prob_test < 0.5, 1-y_pred_prob_test, y_pred_prob_test)))

# Plot decision-boundaries
fig, ax = plot_data(X_train, y_train, X_test, title = 'Lines showing $P(y_{n} = 1 | x_{n}, X, y)$')
fig, ax = decision_boundary(model, fig, ax, labels = True)

# 5. (0 pts)

Finally, lets apply all four of our implemented classifiers to a real world dataset about breast cancer. The [UCI ML Breast Cancer Wisconsin (Diagnostic)](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)) dataset contains characteristics of the cell nuclei, which can be used to predict whether a tumor is malignant or benign.

First, lets import the dataset and split it into a train- and test set.

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the breast cancer dataset
X,y = load_breast_cancer(return_X_y=True)
X = X[:,:10] # Only looking at the first 10 features (the mean values)
X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print('Training set:\t', X_train.shape,  y_train.shape)
print('Test set:\t', X_test.shape,  y_test.shape)

Now we set the parameters for all of our classifiers and train each one of them and report the average train- and test accuracy.

In [ ]:
# Parameters (feel free to change)
learning_rate = 0.05
n_iterations = 1000
C = 10
print_cost = False
K = 5

# Define all models
models = {"KNN":K_Nearest_Neighbors(n_neighbors = K),
          "SVM":SVM(learning_rate, n_iterations, C, print_cost),
          "LR":LogisticRegression(learning_rate, n_iterations, print_cost),
          "GNB":GaussianNaiveBayes()}

# Train all models and report performance
for name, model in models.items():
    print(name)
    model.fit(X_train, y_train)
    if name == "SVM":
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
    else:
        y_pred_train,_ = model.predict(X_train)
        y_pred_test,_ = model.predict(X_test)   
    print("\tTrain accuracy: %.3f %%" %(np.mean(y_pred_train == y_train)*100))
    print("\tTest accuracy: %.3f %%" %(np.mean(y_pred_test == y_test)*100))